In [ ]:
#download Casia WebFace  REPLACE WITH IMPORTS FROM DATASET DIRECTORY
import gdown
file_id = '1KxNCrXzln0lal3N4JiYl9cFOIhT78y1l'
gdown.download(f'https://drive.google.com/uc?id={file_id}', quiet=False)

In [ ]:
!pip install mxnet

In [ ]:
#Mention: NOW WE NEED TO LOCATE THE FILES OF MXNET LIBRARY AND MANUALLY CHANGE THE np.bool to bool to make things work
#File location
..python3.10/dist-packages/mxnet/numpy/utils.py

In [ ]:
import zipfile

with zipfile.ZipFile('/content/faces_webface_112x112.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/Casia')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import pickle
import tensorflow_hub as hub
import random
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
import cv2

# !rm -rf /content/.cache && rm -rf /tmp/*
# from google.colab import drive

# drive.mount('/content/drive')

# !cp /content/drive/MyDrive/FaceData/data_test.pkl /content
# !cp /content/drive/MyDrive/FaceData/data_train.pkl /content

# drive.flush_and_unmount()

# with open('data_test.pkl', 'rb') as f:
#     test_df = pickle.load(f)

# with open('data_train.pkl', 'rb') as f:
#     train_df = pickle.load(f)

# x_train = train_df['image']
# y_train = train_df['person']

# x_test = test_df['image']
# y_test = test_df['person']

# def remove_extra_dim(image):
#     try:
#         return np.squeeze(image, axis=0)  # Efficient removal if size 1
#     except ValueError:
#         # Extra dimension not found, return the original data
#         print('failed')
#         return image

# Loop through each image in the Series
# for index, image in x_train.items():  # Access elements using items()
#     x_train.loc[index] = remove_extra_dim(image)  # Modify Series in-place

# for index, image in x_test.items():  # Access elements using items()
#     x_test.loc[index] = remove_extra_dim(image)  # Modify Series in-place

# x_train[0].shape

# x_train.shape

# classes = set()
# for i in y_train:
#   classes.add(i)

# classes = list(classes)
# classes

def data_generator(batch_size=120):

  dataDIR = '/content/Casia/faces_webface_112x112/'
  rec_file = dataDIR+'train.rec'
  idx_file = dataDIR+'train.idx'

  with open('face_data_mapping.pkl','rb') as f:
    label_wise_sample = pickle.load(f)

  while True:
    a = []
    p = []
    n = []
    for _ in range(batch_size):
      found = True
      while(found):
        pos_neg = random.sample(list(label_wise_sample.keys()), 2)
        try:
          positive_samples = random.sample((label_wise_sample[pos_neg [0]]), 2)
          negative_sample= random.sample((label_wise_sample[pos_neg [1]]),1)
          found = False
        except:
          continue

      record = mx.recordio.MXIndexedRecordIO(idx_file, rec_file, 'r')
      label_wise_index = dict()
      a_img_ind = positive_samples[0]
      p_img_ind = positive_samples[1]
      n_img_ind = negative_sample[0]

      images = []

      image_record = record.read_idx(a_img_ind)
      headerA, imageA = mx.recordio.unpack_img(image_record)

      images.append(imageA)

      image_record = record.read_idx(p_img_ind)
      headerP, imageP = mx.recordio.unpack_img(image_record)

      images.append(imageP)


      image_record = record.read_idx(n_img_ind)
      headerN, imageN = mx.recordio.unpack_img(image_record)

      images.append(imageN)

      processedImages = []

      for i in images:
        image = i[:,:,::-1]
        image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        image = np.asarray(image)
        image = image.astype('float32') / 255.0
        processedImages.append(image)


      a.append(processedImages[0])
      p.append(processedImages[1])
      n.append(processedImages[2])

    yield ([np.array(a), np.array(p), np.array(n)], np.zeros((batch_size, 1)).astype("float32"))
